# Data acquisition in arbok

### Arbok module imports

In [1]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.adb import ADB
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

2023-09-18 11:05:12,411 - qm - INFO     - Starting session: cec815be-f7ee-4a67-a3ff-bce5711755d3


In [2]:
from arbok.core.program import Program
from arbok.core.sample import Sample

Instanciating sample and measurement objects

In [3]:
aurora = Sample('aurora', aurora_qua_config)
ESR_search = Program('ESR_search', sample = aurora)

In [4]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.control.adb import ADB
from arbok.samples.aurora.control.rabi import Rabi
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

Creating Subsequences and adding them to the measurement object `ESR_search`

In [5]:
init6 = InitOdd6dot('init6')
adb6 = ADB('adb6')
read6 = PSB6dotRead('read6')

In [6]:
ESR_search.add_subsequence(init6)
ESR_search.add_subsequence(adb6)
ESR_search.add_subsequence(read6)

In [7]:
ESR_search.adb6.chirp_rate()

21000

# Running the program in the quantify framework

In [8]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl
from quantify_core.visualization.pyqt_plotmon import PlotMonitor_pyqt

Specifying the path for the database containing results

In [9]:
dh.set_datadir()

Data will be saved in:
C:\Users\labuser\quantify-data


Next we connect the physical OPX+ instrument to our program ... 

In [10]:
ESR_search.connect_OPX(host_ip= '192.168.0.3')

2023-09-18 11:05:14,134 - qm - INFO     - Performing health check
2023-09-18 11:05:14,177 - qm - INFO     - Health check passed
2023-09-18 11:05:16,803 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1694999505283
2023-09-18 11:05:16,805 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1694999505283 is canceling running job 1684735462153.


Instanciating `MeasurementControl`

In [11]:
meas_ctrl = MeasurementControl('meas_ctrl')

Creating and connecting a `PlotMonitor` to our `MeasurementControl`

In [12]:
plotmon = PlotMonitor_pyqt("plotmon")
meas_ctrl.instr_plotmon(plotmon.name)

C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots is deprecated. Use "qcodes_loop.plots" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots.colors is deprecated. Use "qcodes_loop.plots.colors" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\qcodes\plots\pyqtgraph.py:17: QCoDeSDeprecationWarning: The qcodes.utils.qt_helpers module is deprecated. Use "qcodes_loop.utils.qt_helpers" as an alternative.
  import qcodes.utils.qt_helpers
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:19: QCoDeSDeprecationWarning: The qcodes.plots.pyqtgraph is deprecated. Use "qcodes_loop.plots

## How to measure: adiabatic ESR search

So far so good, now fully set up our adiabatic ESR search. We have a init, control and readout sequence. Now we have to define parameters to sweep and to measure.

Defining the frequency sweep list:

In [13]:
import numpy as np

In [14]:
f_detune=np.arange(25e6,80e6,0.1e6, dtype = int)

Defining the parameters to be swept and adding setpoints to them. Also parameters to be recorded are added. In this case the measured state of SET 1 and 2

In [15]:
ESR_search.settables = [
    ESR_search.adb6.IfQ1]
ESR_search.setpoints_grid = [
    f_detune]
ESR_search.gettables = [
    ESR_search.read6.diff_set1_state,
    ESR_search.read6.diff_set2_state]

Just defined values are automatically added to `meas_ctrl` by running `prepare_meas_ctrl`

In [16]:
ESR_search.prepare_gettables()

None


In [17]:
ESR_search.prepare_meas_ctrl(meas_ctrl)

<Arrays, shape: (550,)>
<Arrays, shape: (550,)>


... and run the job! Dont worry, the program will only run the first iteration and wait for us to retreive the result and only continue after that.

In [18]:
#ESR_search.run()

If all this is done, the actual measurement can be run! Check the taskbar for your plot monitor and see the buffered results come in.

In [19]:
#meas_ctrl.run("adb_ESR_search", soft_avg=100)

## How to measure: Rabi Chevron

Sometimes we want to measure two dimensional parameter sweeps. This works very similarly to the 1D case. We just have to add a second settable a corresponding value array to the setpoints.

But lets first create the sequence:

In [20]:
RabiChevron = Program('RabiChevron', sample = aurora)
rabi6 = Rabi('rabi6')
RabiChevron.add_subsequence(init6)
RabiChevron.add_subsequence(rabi6)
RabiChevron.add_subsequence(read6)

In [21]:
f_detune=np.arange(32.2e6,34.2e6,0.05e6, dtype = int)
print(len(f_detune))
t_list = np.arange(4,30e3/4,200, dtype = int)
print(len(t_list))

40
38


In [22]:
RabiChevron.settables = [
    RabiChevron.rabi6.IfQ1,
    RabiChevron.rabi6.tRabi]
RabiChevron.setpoints_grid = [
    f_detune,
    t_list]
RabiChevron.gettables = [
    RabiChevron.read6.diff_set1_state,
    RabiChevron.read6.diff_set2_state]

RabiChevron.connect_OPX(host_ip= '192.168.0.3')
RabiChevron.prepare_meas_ctrl(meas_ctrl)

2023-09-18 11:05:18,918 - qm - INFO     - Performing health check
2023-09-18 11:05:18,939 - qm - INFO     - Health check passed
<Arrays, shape: (1520,)>
<Arrays, shape: (1520,)>
<Arrays, shape: (1520,)>
<Arrays, shape: (1520,)>


In [23]:
rabi_chev_program = RabiChevron.get_program()

Adding qua <class 'int'> variable for IfQ1
Adding qua <class 'int'> variable for tRabi
Adding qua sweep loop for tRabi
Adding qua sweep loop for IfQ1


In [24]:
RabiChevron.prepare_meas_ctrl(meas_ctrl)

<Arrays, shape: (1520,)>
<Arrays, shape: (1520,)>
<Arrays, shape: (1520,)>
<Arrays, shape: (1520,)>


In [25]:
RabiChevron.run(rabi_chev_program)

2023-09-18 11:05:22,646 - qm - INFO     - Sending program to QOP for compilation
2023-09-18 11:05:57,641 - qm - INFO     - Executing program


In [26]:
meas_ctrl.run("RabiChevron", soft_avg=100)

Starting batched measurement...
Iterative settable(s) [outer loop(s)]:
	 --- (None) --- 
Batched settable(s):
	 IfQ1, tRabi 
Batch size limit: 1520

(40, 38)
FETC 353 1520 diff_set1_state
WAIT 353 1520
WAIT 633 1520
WAIT 991 1520
WAIT 1206 1520
READ 1520 1520
2023-09-18 11:05:59,460 - qm - WARNING  - Fetching single result will always return the single value
Buffersize (1520,)


ValueError: ('Shape of output is not consistent with setpoints. Output is shape (40, 38) and setpoints are shape (1520, 1520)', 'getting diff_set1_state')

In [ ]:
tuple([np.prod( [len(x) for x in RabiChevron.setpoints_grid])])

In [ ]:
for x in ESR_search.setpoints_grid:
    print(len(x))

In [ ]:
a = [ len(x) for x in RabiChevron.setpoints_grid]

In [ ]:
np.prod(a)

In [ ]:
u = [len(x) for x in RabiChevron.setpoints_grid]

In [ ]:
[np.prod(u)]